In [18]:
import pandas as pd
df = pd.read_csv("df_clean.csv")


### Features prévues


1. Âge du bâtiment

Formule : 2024 - YearBuilt

Pourquoi : L’année brute n’aide pas vraiment le modèle. L’âge donne une idée de la vétusté, donc de la qualité probable de l’isolation.

Info : Les bâtiments anciens consomment souvent plus (SiteEUI plus élevé).

2. Nombre d’usages du bâtiment

Formule : compter le nombre d’usages présents dans ListOfAllPropertyUseTypes.

Pourquoi : Un bâtiment multi-usage (ex : commerce + bureau) est plus compliqué énergétiquement qu’un bâtiment mono-usage.

Info : Les combinaisons d’usages changent les comportements de consommation.

3. Ratio parking / surface totale

Formule : PropertyGFAParking / PropertyGFATotal

Pourquoi : Le parking n’est presque pas chauffé/climatisé. S’il occupe une grande surface, ça influence la consommation rapportée au m².

Info : Deux grands bâtiments peuvent sembler différents juste à cause d’un parking énorme.

4. Part de l’usage principal

Formule : LargestPropertyUseTypeGFA / PropertyGFATotal

Pourquoi : Si un bâtiment est composé à 90 % de bureaux ou seulement à 40 %, ce n’est pas la même chose en termes d’énergie.

Info : La surface dominante influence beaucoup la consommation globale.

1. Age du batiment

In [19]:
df["Agedubatiment"] = 2024 - df["YearBuilt"]


In [20]:
df.columns


Index(['BuildingType', 'PrimaryPropertyType', 'ZipCode', 'CouncilDistrictCode',
       'Neighborhood', 'Latitude', 'Longitude', 'YearBuilt',
       'NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
       'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SecondLargestPropertyUseTypeGFA', 'ThirdLargestPropertyUseType',
       'ThirdLargestPropertyUseTypeGFA', 'YearsENERGYSTARCertified',
       'ENERGYSTARScore', 'SiteEUIWN(kBtu/sf)', 'SteamUse(kBtu)',
       'Agedubatiment'],
      dtype='object')

2. Nombre d’usages du bâtiment

In [21]:
def count_usages(value):
    if pd.isna(value):
        return 0
    else:
        return len(value.split(','))

df["UsageCount"] = df["ListOfAllPropertyUseTypes"].apply(count_usages)

df["UsageCount"].value_counts()


UsageCount
1     693
2     485
3     197
4      79
5      40
6      18
7       4
9       2
11      1
13      1
8       1
Name: count, dtype: int64

3. Ratio parking / surface totale


In [25]:
df["ParkingRatio"] = df["PropertyGFAParking"] / df["PropertyGFATotal"]


df["ParkingRatio"].describe()


count    1521.000000
mean        0.610464
std         8.270967
min      -149.512892
25%         0.567631
50%         0.644264
75%         0.932823
max       143.691492
Name: ParkingRatio, dtype: float64

4. Part de l’usage principal


In [26]:
df["PrimaryUseRatio"] = df["LargestPropertyUseTypeGFA"] / df["PropertyGFATotal"]

df["PrimaryUseRatio"].describe()


count    1521.000000
mean        1.212645
std         6.230674
min       -52.818912
25%         0.795634
50%         0.925973
75%         1.138282
max       188.454730
Name: PrimaryUseRatio, dtype: float64